In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import math
import json
import h5py
from tqdm import tqdm
import os
'''
from google.colab import drive
drive.mount('/content/drive')
'''

Mounted at /content/drive


In [ ]:
def extract_defensive_line_features(freeze_frame):
    if not freeze_frame:
        return None, None
    defenders = [p for p in freeze_frame if not p.get("teammate", False)]
    if not defenders:
        return None, None
    line_labels = assign_defensive_lines(defenders)
    return compute_line_gaps(defenders, line_labels)

def extract_onball_attacker_position(freeze_frame):
    if not isinstance(freeze_frame, list):
        return None
    for player in freeze_frame:
        if player.get("actor", False):
            return player.get("location", None)
    return None

def compute_ball_direction_vector(event):
    freeze_frame = event.get("freeze_frame")
    end_x = event.get("end_x")
    end_y = event.get("end_y")

    if freeze_frame is None or end_x is None or end_y is None:
        return (0.0, 0.0)  # fallback vector

    start_pos = extract_onball_attacker_position(freeze_frame)
    if start_pos is None:
        return (0.0, 0.0)

    dx = end_x - start_pos[0]
    dy = end_y - start_pos[1]

    return (dx, dy)


def assign_defensive_lines(defenders, x_threshold=3.0, y_tolerance=2.0):
    if not defenders:
        return []

    x_coords = np.array([p["location"][0] for p in defenders])
    y_coords = np.array([p["location"][1] for p in defenders])

    # 1. Central defender extraction
    median_y = np.median(y_coords)
    central_def = np.where(np.abs(y_coords - median_y) <= y_tolerance)[0]
    if len(central_def) == 0:
        for p in defenders:
            p["line"] = -1
        return defenders

    # 2. X-coordinate alignment of central defenders
    central_x_coords = sorted(x_coords[central_def])

    assigned = np.full(len(defenders), -1, dtype=int)
    current_line = 0

    for cx in central_x_coords:
        for i in range(len(defenders)):
            if assigned[i] == -1 and abs(x_coords[i] - cx) <= x_threshold:
                assigned[i] = current_line
        current_line += 1

    for i in range(len(defenders)):
        if assigned[i] == -1:
            min_dist = float('inf')
            best_line = current_line
            for j in range(len(defenders)):
                if assigned[j] != -1:
                    dist = abs(x_coords[i] - x_coords[j])
                    if dist < min_dist and dist <= x_threshold:
                        min_dist = dist
                        best_line = assigned[j]
            assigned[i] = best_line if min_dist != float('inf') else current_line
            if best_line == current_line:
                current_line += 1

    for idx, p in enumerate(defenders):
        p["line"] = assigned[idx]

    return defenders

def compute_line_gaps(defenders, line_labels=None, use_median=False):
    if not defenders:
        return None, None

    if line_labels is None:
        line_labels = [p.get("line", -1) for p in defenders]


    line_to_x = collections.defaultdict(list)
    line_to_y = collections.defaultdict(list)

    for d, label in zip(defenders, line_labels):
        if isinstance(label, (int, float)) and label != -1:
            x, y = d["location"]
            line_to_x[int(label)].append(x)
            line_to_y[int(label)].append(y)

    if not line_to_x:
        return None, None

    # line_gap
    center_fn = np.median if use_median else np.mean
    centers_x = {lbl: center_fn(xs) for lbl, xs in line_to_x.items()}
    sorted_labels = sorted(centers_x, key=lambda L: centers_x[L])

    line_gap = None
    if len(sorted_labels) >= 2:
        gaps = [
            centers_x[sorted_labels[i + 1]] - centers_x[sorted_labels[i]]
            for i in range(len(sorted_labels) - 1)
        ]
        line_gap = max(gaps) if gaps else None

    # in_line_gap
    spans_y = []
    for ys in line_to_y.values():
        if len(ys) >= 2:
            spans_y.append(max(ys) - min(ys))
        else:
            spans_y.append(0.0)
    in_line_gap = max(spans_y) if spans_y else None

    return line_gap, in_line_gap


def detect_defensive_collapse_onball(
    freeze_frame_current,
    freeze_frame_after,
    w1=0.01,
    w2=0.03,
    w3=0.05,
    exclude_keeper=True
):
    if not isinstance(freeze_frame_current, list) or not isinstance(freeze_frame_after, list):
        return 0.0

    defenders_current = [
        p for p in freeze_frame_current
        if (not p.get("teammate", False)) and (not exclude_keeper or not p.get("keeper", False))
    ]

    defenders_after = [
        p for p in freeze_frame_after
        if (not p.get("teammate", False)) and (not exclude_keeper or not p.get("keeper", False))
    ]

    num_current = len(defenders_current)
    num_after = len(defenders_after)
    diff = num_current - num_after

    if diff >= 3:
        return float(w3)
    elif diff == 2:
        return float(w2)
    elif diff == 1:
        return float(w1)
    else:
        return 0.0

In [ ]:
def detect_defensive_collapse_offball(
    freeze_frame_current, freeze_frame_after, ball_direction_vector,
    line_gap_threshold=5.0, in_line_gap_threshold=5.0, cos_sim_thr=0.8
):
    if not freeze_frame_current or not freeze_frame_after:
        return False

    attackers_current = {
        (p.get("player_id") or p.get("actor_id")): p["location"]
        for p in freeze_frame_current
        if p.get("teammate", False) and not p.get("actor", False) and "location" in p
    }
    attackers_after = {
        (p.get("player_id") or p.get("actor_id")): p["location"]
        for p in freeze_frame_after
        if p.get("teammate", False) and not p.get("actor", False) and "location" in p
    }

    def cosine_similarity(vec1, vec2):
        if not isinstance(vec1, (list, tuple)) or not isinstance(vec2, (list, tuple)):
            return 0.0
        if len(vec1) < 2 or len(vec2) < 2:
            return 0.0
        dot = vec1[0]*vec2[0] + vec1[1]*vec2[1]
        norm1 = math.sqrt(vec1[0]**2 + vec1[1]**2)
        norm2 = math.sqrt(vec2[0]**2 + vec2[1]**2)
        return (dot / (norm1 * norm2)) if (norm1 and norm2) else 0.0

    moved_similar_dir = False
    for pid, pos_before in attackers_current.items():
        pos_after = attackers_after.get(pid)
        if not pos_after:
            continue
        dx = pos_after[0] - pos_before[0]
        dy = pos_after[1] - pos_before[1]
        if dx == 0 and dy == 0:
            continue  # 정지
        sim = cosine_similarity((dx, dy), ball_direction_vector)
        if sim >= float(cos_sim_thr):
            moved_similar_dir = True
            break

    if not moved_similar_dir:
        return False

    gap_current, in_gap_current = extract_defensive_line_features(freeze_frame_current)
    gap_after,   in_gap_after   = extract_defensive_line_features(freeze_frame_after)
    if gap_current is None or gap_after is None:
        return False

    return (
        ((gap_after or 0)   - (gap_current or 0)   >= float(line_gap_threshold)) or
        ((in_gap_after or 0) - (in_gap_current or 0) >= float(in_line_gap_threshold))
    )

In [ ]:
def check_action_chain_result(events, start_index, nr_actions=10):
    GOAL_TYPES = {'shot', 'shot_freekick', 'shot_penalty'}
    shot, goal = False, False

    end = min(start_index + 1 + nr_actions, len(events))

    for i in range(start_index + 1, end):
        ev = events[i]
        at = (ev.get('action_type') or '').lower()
        res = (ev.get('result') or '').lower()

        if at in GOAL_TYPES:
            shot = True
            if res == 'success':
                goal = True
                break

    return {"shot": shot, "goal": goal}

In [ ]:
def compute_on_offball_features(
    events,
    nr_actions=10,
    # on-ball weighted value
    w1=0.01, w2=0.03, w3=0.05,
    on_shot_bonus=0.02,
    on_goal_bonus=0.06,
    exclude_keeper=True,
    line_gap_threshold=5.0,
    in_line_gap_threshold=5.0,
    cos_sim_thr=0.8,
    # off-ball weighted value
    off_base=0.02,
    off_goal_bonus=0.03
):
    results = []

    for i, current in enumerate(events):
        next_ev = events[i + 1] if i + 1 < len(events) else None

        out = {
            "index": current.get("index"),
            "event_uuid": current.get("event_uuid"),
            "onball_collapse": False,
            "offball_collapse": False,
            "contributed_shot": False,
            "contributed_goal": False,
            "onball_contribution_score": 0.0,
            "offball_contribution_score": 0.0,
        }

        if (
            next_ev is None or
            current.get("game_id") != next_ev.get("game_id") or
            current.get("period_id") != next_ev.get("period_id")
        ):
            results.append(out)
            continue

        ff_cur = current.get("freeze_frame")
        ff_nxt = next_ev.get("freeze_frame")
        if not isinstance(ff_cur, list) or not isinstance(ff_nxt, list):
            results.append(out)
            continue

        if (current.get("action_type") or "").lower() in {"dribble"}:
            on_val = float(
                detect_defensive_collapse_onball(
                    ff_cur, ff_nxt,
                    w1=w1, w2=w2, w3=w3,
                    exclude_keeper=exclude_keeper
                )
            )
            if on_val > 0.0:
                out["onball_collapse"] = True
                out["onball_contribution_score"] = on_val

        if (current.get("action_type") or "").lower() in {"pass", "dribble"}:
            ball_vec = compute_ball_direction_vector(current)
            off_flag = bool(
                detect_defensive_collapse_offball(
                    ff_cur, ff_nxt, ball_vec,
                    line_gap_threshold=line_gap_threshold,
                    in_line_gap_threshold=in_line_gap_threshold,
                    cos_sim_thr=cos_sim_thr
                )
            )
            if off_flag:
                out["offball_collapse"] = True
                out["offball_contribution_score"] = float(off_base)

        if out["onball_collapse"] or out["offball_collapse"]:
            contrib = check_action_chain_result(events, i, nr_actions=nr_actions)
            shot_flag = bool(contrib.get("shot", False))
            goal_flag = bool(contrib.get("goal", False))
            out["contributed_shot"] = shot_flag
            out["contributed_goal"] = goal_flag

            if out["onball_collapse"] and out["onball_contribution_score"] > 0.0:
                if goal_flag:
                    out["onball_contribution_score"] += float(on_goal_bonus)
                elif shot_flag:
                    out["onball_contribution_score"] += float(on_shot_bonus)

            if out["offball_collapse"] and goal_flag:
                out["offball_contribution_score"] += float(off_goal_bonus)

        results.append(out)

    return pd.DataFrame(results)


In [ ]:
'''
merged_path = "/content/drive/MyDrive/Data/Processed/euro2020_spadl_merged.h5"
save_path = "/content/drive/MyDrive/Data/Processed/euro2020_spadl_enriched.h5"

with pd.HDFStore(save_path, mode="w") as store:
    pass

with pd.HDFStore(merged_path, mode="r") as store:
    match_keys = [k for k in store.keys() if k.startswith("/actions/")]
    match_ids = [int(k.split("/")[-1]) for k in match_keys]

for match_id in match_ids:
    print(f"Processing match: {match_id}")

    with pd.HDFStore(merged_path, mode="r") as store:
        df = store[f"/actions/{match_id}"].copy()

    df = df.sort_values(["period_id", "seconds"]).reset_index(drop=True)

    feature_df = compute_on_offball_features(df.to_dict("records"))
    feature_df = feature_df.drop_duplicates(subset=["event_uuid"])

    # Merge
    event_df = df.merge(feature_df.drop(columns=["index"]), on="event_uuid", how="left")

    # Save
    with pd.HDFStore(save_path, mode="a") as store:
        store[f"/actions/{match_id}"] = event_df

print("All Matches Saved (overwrite + no duplication)")

'''

Processing match: 3788741


/usr/local/lib/python3.11/dist-packages/tables/path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '3788741'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipython-input-3415647196.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['event_id', 'player_name', 'team_name', 'bodypart', 'action_type',
       'result', 'event_uuid', 'visible_area', 'freeze_frame'],
      dtype='object')]

  store[f"/actions/{match_id}"] = event_df


Processing match: 3788742


/usr/local/lib/python3.11/dist-packages/tables/path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '3788742'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipython-input-3415647196.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['event_id', 'player_name', 'team_name', 'bodypart', 'action_type',
       'result', 'event_uuid', 'visible_area', 'freeze_frame'],
      dtype='object')]

  store[f"/actions/{match_id}"] = event_df


Processing match: 3788743


...

Processing match: 3795221


/usr/local/lib/python3.11/dist-packages/tables/path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '3795221'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipython-input-3415647196.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['event_id', 'player_name', 'team_name', 'bodypart', 'action_type',
       'result', 'event_uuid', 'visible_area', 'freeze_frame'],
      dtype='object')]

  store[f"/actions/{match_id}"] = event_df


Processing match: 3795506


/usr/local/lib/python3.11/dist-packages/tables/path.py:146: NaturalNameWarning: object name is not a valid Python identifier: '3795506'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipython-input-3415647196.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['event_id', 'player_name', 'team_name', 'bodypart', 'action_type',
       'result', 'event_uuid', 'visible_area', 'freeze_frame'],
      dtype='object')]

  store[f"/actions/{match_id}"] = event_df


All Matches Saved (overwrite + no duplication)
